# <font color="#418FDE" size="6.5" uppercase>**Model Export**</font>

>Last update: 20260130.
    
By the end of this Lecture, you will be able to:
- Describe the purpose and capabilities of torch.export in PyTorch 2.10.0. 
- Export a trained nn.Module to an exported program suitable for deployment or further compilation. 
- Inspect and validate exported models to ensure they produce consistent outputs with the original model. 


## **1. Core Export Concepts**

### **1.1. Eager vs Exported**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_09/Lecture_A/image_01_01.jpg?v=1769774425" width="250">



>* Eager mode runs Python code dynamically each call
>* Great for experimentation, but weak for stable deployment

>* Exports eager models into static, Python-free graphs
>* Creates stable contracts for deployment and tooling

>* Eager mode favors flexible experimentation and debugging
>* Exported models trade flexibility for stability and deployment



In [ ]:
#@title Python Code - Eager vs Exported

# This script compares eager and exported style behavior.
# It uses simple functions to illustrate static computation.
# Focus is on concepts not real torch.export internals.
# Example import for numerical work if needed.
import math

# Define a simple eager style Python function.
def eager_square_sum(x_list):
    total = 0
    for value in x_list:
        total += value * value
    return total

# Define a tiny exported style representation.
class ExportedSquareSum:
    def __init__(self, length):
        self.length = length

    def run(self, x_list):
        if len(x_list) != self.length:
            raise ValueError("Unexpected input length for program")
        total = 0
        for value in x_list:
            total += value * value
        return total

# Helper function to print a short separator line.
def print_separator(title):
    print("\n---", title, "---")

# Prepare a small deterministic input list.
input_values = [1.0, 2.0, 3.0]

# Validate the input length before using it.
if len(input_values) <= 0:
    raise ValueError("Input list must not be empty")

# Run the eager style computation.
eager_result = eager_square_sum(input_values)

# Create an exported style program with fixed length.
exported_program = ExportedSquareSum(length=len(input_values))

# Run the exported style computation.
exported_result = exported_program.run(input_values)

# Show that both approaches give the same numeric result.
print_separator("Eager versus exported results")
print("Input values:", input_values)
print("Eager result:", eager_result)
print("Exported result:", exported_result)

# Demonstrate that exported program expects fixed length.
short_input = [1.0, 2.0]

# Try running with wrong length and handle the error.
print_separator("Exported length check behavior")
try:
    _ = exported_program.run(short_input)
except ValueError as exc:
    print("Exported program error:", str(exc))

# Final confirmation that eager still runs with any length.
print("Eager with short input:", eager_square_sum(short_input))




### **1.2. Graph Capture Constraints**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_09/Lecture_A/image_01_02.jpg?v=1769774468" width="250">



>* Export turns flexible Python models into static graphs
>* Graphs allow optimization but restrict Python features

>* Control flow and shapes must be predictable
>* Symbolic tensor logic works; opaque Python breaks export

>* Export prefers pure tensor math without side effects
>* Move logging and I/O outside the exported graph



In [ ]:
#@title Python Code - Graph Capture Constraints

# This script illustrates graph capture constraints simply.
# We simulate export friendly and unfriendly model behavior.
# Focus on predictable control flow and side effects.

# Required standard imports only, no extra installations.
# !pip install torch.

# Import random for deterministic branching demonstration.
import random
# Import os for potential environment based configuration.
import os

# Set deterministic seed for reproducible random behavior.
random.seed(0)
# Define a tiny tensor like structure using Python lists.
small_tensor = [1.0, 2.0, 3.0]

# Define a function that mimics pure tensor computation.
def pure_computation(tensor_values, scale):
    # Use predictable loop based only on tensor length.
    result = []
    # Iterate over values and apply simple scaling.
    for value in tensor_values:
        result.append(value * scale)
    # Return new list without side effects.
    return result

# Define a function with Python side effects and randomness.
def side_effect_computation(tensor_values):
    # Mutate external state and use random branching.
    global small_tensor
    # Random choice makes control flow unpredictable.
    if random.random() > 0.5:
        small_tensor.append(99.0)
    # Return original list without clear relation.
    return tensor_values

# Demonstrate predictable, export friendly style behavior.
scale_factor = 2.0
# Call pure computation that would be graph friendly.
pure_output = pure_computation(small_tensor, scale_factor)

# Demonstrate unpredictable, export unfriendly behavior.
# Call side effect computation that changes global state.
side_output = side_effect_computation(small_tensor)

# Show original tensor values before any mutation.
print("Initial small_tensor values:", [1.0, 2.0, 3.0])
# Show output from pure computation, stable and predictable.
print("Pure computation output:", pure_output)

# Show tensor after potential mutation from side effects.
print("small_tensor after side_effect:", small_tensor)
# Show side effect function output for comparison.
print("Side effect computation output:", side_output)

# Explain why pure computation is export friendly.
print("Pure path uses fixed loops and no side effects.")
# Explain why side effect path breaks graph capture.
print("Side effect path mutates globals and uses randomness.")

# Final confirmation that shapes remain predictable for pure path.
print("Length of pure_output is", len(pure_output))




### **1.3. Supported export patterns**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_09/Lecture_A/image_01_03.jpg?v=1769774616" width="250">



>* Export supports many standard deep learning architectures
>* Handles tensor-based control flow as unified graph

>* Export handles dynamic shapes and flexible inputs
>* One exported model serves many input sizes efficiently

>* Export supports multi-stage pipelines across devices
>* Preprocessing and postprocessing can live inside exports



In [ ]:
#@title Python Code - Supported export patterns

# This script shows supported export patterns simply.
# We simulate export ideas without real PyTorch usage.
# Focus on control flow and dynamic shapes conceptually.

# Required external libraries would be installed here.
# !pip install torch torchvision torchaudio.

# Import standard modules for numerical work.
import math
import random
import os

# Set deterministic random seed for reproducibility.
random.seed(0)

# Define a simple feedforward like model using functions.
def simple_feedforward(x_list):
    # Apply a linear like transform to each element.
    return [2.0 * x + 1.0 for x in x_list]

# Define a model with tensor like control flow branching.
def branching_model(x_list, threshold):
    # Choose path depending on average value condition.
    avg = sum(x_list) / float(len(x_list))
    if avg > threshold:
        # If high average, apply feedforward twice.
        return simple_feedforward(simple_feedforward(x_list))
    else:
        # If low average, apply feedforward once only.
        return simple_feedforward(x_list)

# Define a loop based sequence style processing model.
def loop_sequence_model(x_list, max_steps):
    # Start with an empty list for outputs.
    outputs = []
    step = 0
    # Iterate until max steps or list end reached.
    while step < max_steps and step < len(x_list):
        value = x_list[step]
        outputs.append(value * 0.5)
        step += 1
    return outputs

# Define a dynamic shape friendly wrapper function.
def dynamic_shape_pipeline(x_list, threshold, max_steps):
    # Validate input is a non empty list.
    if not isinstance(x_list, list) or len(x_list) == 0:
        raise ValueError("x_list must be non empty list")
    # Apply branching model to full list first.
    branch_out = branching_model(x_list, threshold)
    # Then apply loop model to branching output.
    loop_out = loop_sequence_model(branch_out, max_steps)
    return loop_out

# Define a tiny feature extraction like stage.
def feature_extractor(raw_values):
    # Normalize values using simple min max scaling.
    min_v = min(raw_values)
    max_v = max(raw_values)
    if max_v == min_v:
        return [0.0 for _ in raw_values]
    return [(v - min_v) / (max_v - min_v) for v in raw_values]

# Define a tiny prediction head consuming extracted features.
def prediction_head(features, bias):
    # Compute a simple score using sum and bias.
    score = sum(features) + bias
    # Apply thresholding to get binary decision.
    return 1 if score > 1.5 else 0

# Define a pipeline combining two exported style stages.
def two_stage_pipeline(raw_values, threshold, max_steps, bias):
    # First stage performs feature extraction step.
    feats = feature_extractor(raw_values)
    # Second stage runs dynamic shape pipeline.
    processed = dynamic_shape_pipeline(feats, threshold, max_steps)
    # Final stage applies prediction head decision.
    decision = prediction_head(processed, bias)
    return decision, feats, processed

# Prepare two different dynamic length input examples.
example_short = [0.2, 0.4, 0.6]
example_long = [0.1, 0.3, 0.5, 0.7, 0.9]

# Run pipeline on short example with parameters.
short_decision, short_feats, short_proc = two_stage_pipeline(
    example_short,
    threshold=0.4,
    max_steps=5,
    bias=0.1,
)

# Run pipeline on long example with different parameters.
long_decision, long_feats, long_proc = two_stage_pipeline(
    example_long,
    threshold=0.3,
    max_steps=3,
    bias=0.2,
)

# Print framework placeholder version information.
print("PyTorch version placeholder for export concepts.")

# Show that dynamic shapes still follow same pipeline.
print("Short input length and decision:", len(example_short), short_decision)

# Show processed length for short dynamic example.
print("Short processed length:", len(short_proc))

# Show that longer input reuses same computation pattern.
print("Long input length and decision:", len(example_long), long_decision)

# Show processed length for long dynamic example.
print("Long processed length:", len(long_proc))

# Confirm branching behavior by printing one internal average.
print("Average of short features:", sum(short_feats) / len(short_feats))




## **2. Exporting Models with torchexport**

### **2.1. Defining example inputs**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_09/Lecture_A/image_02_01.jpg?v=1769774690" width="250">



>* Example inputs must mirror real deployment data
>* Accurate examples ensure complete, robust exported models

>* Capture real input variability and structure carefully
>* Match example inputs to true production data flows

>* Include edge cases and hardware limits in examples
>* Use challenging scenarios to expose hidden model issues



In [ ]:
#@title Python Code - Defining example inputs

# This script shows defining example inputs.
# We use TensorFlow to mimic model behavior.
# Focus on shapes and simple consistency checks.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import random
import numpy as np

# Import TensorFlow and Keras layers.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Set deterministic seeds for reproducibility.
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Define a tiny image classifier model.
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(4, (3, 3), activation="relu")(inputs)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

# Compile model with simple settings.
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

# Create a realistic example input batch.
batch_size = 2
height, width, channels = 28, 28, 1
example_images = np.random.rand(
    batch_size,
    height,
    width,
    channels,
).astype("float32")

# Validate example input shape and dtype.
print("Example batch shape:", example_images.shape)
print("Example batch dtype:", example_images.dtype)

# Run the model once on example inputs.
example_outputs = model.predict(example_images, verbose=0)

# Check that output shape matches expectations.
print("Output batch shape:", example_outputs.shape)

# Show first prediction vector length only.
print("First prediction length:", len(example_outputs[0]))

# Demonstrate a mismatched example input shape.
wrong_example = np.random.rand(batch_size, height, width).astype("float32")

# Try calling model with wrong shape inside try block.
try:
    model.predict(wrong_example, verbose=0)
except Exception as e:
    print("Mismatched example error type:", type(e).__name__)




### **2.2. Exporting Neural Networks**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_09/Lecture_A/image_02_02.jpg?v=1769774856" width="250">



>* Export freezes the trained model into stable form
>* Creates a static graph for optimization and deployment

>* Export splits computation graph from fixed parameters
>* Same graph reused with different tuned parameter sets

>* Exported models must satisfy strict deployment constraints
>* Exporting simplifies, stabilizes, and shapes models for portability



In [ ]:
#@title Python Code - Exporting Neural Networks

# This script shows a tiny neural network export example.
# We simulate exporting by saving weights and structure description.
# Focus is on concept not heavy training or big data.

# Required external installs would be listed here if needed.
# No extra installs are required for this simple example.

# Import standard libraries for numerical work and reproducibility.
import numpy as np
import random as pyrandom

# Import TensorFlow as our simple neural network framework.
import tensorflow as tf

# Set deterministic seeds for reproducible behavior and outputs.
pyrandom.seed(0)
np.random.seed(0)

# Set TensorFlow random seed for deterministic initialization.
tf.random.set_seed(0)

# Print TensorFlow version in one concise line.
print("TensorFlow version:", tf.__version__)

# Create a tiny synthetic dataset for demonstration only.
inputs = np.linspace(-1.0, 1.0, num=8, dtype=np.float32)

# Reshape inputs to match dense layer expected input shape.
inputs = inputs.reshape((-1, 1))

# Create simple targets using a linear relationship with noise.
noise = np.random.normal(loc=0.0, scale=0.05, size=inputs.shape)

# Compute targets as two times input plus small noise term.
targets = 2.0 * inputs + noise.astype(np.float32)

# Define a very small sequential neural network model.
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1,)),
    tf.keras.layers.Dense(units=1, activation=None),
])

# Compile the model with mean squared error loss function.
model.compile(optimizer="sgd", loss="mse")

# Train briefly with silent output to keep logs minimal.
model.fit(inputs, targets, epochs=50, verbose=0)

# Run the trained model on the inputs to get reference outputs.
original_outputs = model.predict(inputs, verbose=0)

# Validate shapes to ensure safe export operations.
assert original_outputs.shape == targets.shape

# Prepare a simple dictionary describing model structure.
export_structure = {
    "input_shape": list(model.input_shape[1:]),
    "output_shape": list(model.output_shape[1:]),
    "layers": ["Dense(1, linear)"],
}

# Extract trained weights and biases from the dense layer.
weights, biases = model.layers[0].get_weights()

# Convert weights and biases to lists for JSON friendly export.
weights_list = weights.tolist()

# Convert biases to list for consistent simple serialization.
biases_list = biases.tolist()

# Bundle structure and parameters into one export artifact object.
export_artifact = {
    "structure": export_structure,
    "weights": weights_list,
    "biases": biases_list,
}

# Rebuild a new model using the exported structure description.
reloaded_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=tuple(export_artifact["structure"]["input_shape"])),
    tf.keras.layers.Dense(units=1, activation=None),
])

# Set the reloaded model weights from the exported parameters.
reloaded_model.layers[0].set_weights([
    np.array(export_artifact["weights"], dtype=np.float32),
    np.array(export_artifact["biases"], dtype=np.float32),
])

# Run the reloaded model on the same inputs for comparison.
reloaded_outputs = reloaded_model.predict(inputs, verbose=0)

# Compute maximum absolute difference between outputs for validation.
max_diff = np.max(np.abs(original_outputs - reloaded_outputs))

# Print a short summary of export and reload consistency.
print("Original outputs shape:", original_outputs.shape)
print("Reloaded outputs shape:", reloaded_outputs.shape)
print("Maximum absolute difference:", float(max_diff))
print("Export structure description:", export_structure)




### **2.3. Persisting Exported Models**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_09/Lecture_A/image_02_03.jpg?v=1769774923" width="250">



>* Save exported models as reusable serialized artifacts
>* Ensures deployed behavior matches tested training environment

>* Use naming, metadata, versions to organize models
>* Track model history for debugging, audits, compliance

>* Persisted exports run across many systems and tools
>* They enable portable, consistent, production-ready deployments



In [ ]:
#@title Python Code - Persisting Exported Models

# This script shows persisting exported style models.
# We simulate export using simple TensorFlow SavedModel.
# Focus is on saving loading and validating outputs.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import pathlib
import random

# Import numpy for small numeric operations.
import numpy as np

# Import tensorflow as lightweight model framework.
import tensorflow as tf

# Set deterministic seeds for reproducibility.
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Print TensorFlow version in one concise line.
print("TensorFlow version:", tf.__version__)

# Define a tiny dense model for demonstration.
model = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=(4,)),
        tf.keras.layers.Dense(3, activation="relu"),
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

# Compile model with simple optimizer and loss.
model.compile(optimizer="adam", loss="mse")

# Create tiny deterministic training data batch.
x_train = np.array([[0.1, 0.2, 0.3, 0.4]], dtype=np.float32)
y_train = np.array([[0.5]], dtype=np.float32)

# Validate shapes before training for safety.
assert x_train.shape == (1, 4)
assert y_train.shape == (1, 1)

# Train for few epochs with silent verbose setting.
model.fit(x_train, y_train, epochs=10, verbose=0)

# Create a small sample input for export testing.
sample_input = np.array([[0.9, 0.8, 0.1, 0.2]], dtype=np.float32)

# Validate sample input shape before inference.
assert sample_input.shape == (1, 4)

# Run original model to get reference prediction.
original_output = model.predict(sample_input, verbose=0)

# Print original model prediction for comparison.
print("Original model output:", original_output.flatten())

# Choose directory path for persisted exported model.
export_dir = pathlib.Path("exported_model_demo")

# Remove existing directory if present for cleanliness.
if export_dir.exists():
    for item in export_dir.iterdir():
        if item.is_file():
            item.unlink()
        else:
            pass

# Save model as a stable SavedModel artifact.
model.export(export_dir)

# Confirm directory exists after saving operation.
print("Saved model directory exists:", export_dir.exists())

# Load model back to simulate deployment environment.
loaded_model = tf.keras.layers.TFSMLayer(str(export_dir), call_endpoint='serving_default')

# Run loaded model on same sample input.
loaded_output = loaded_model(sample_input)

# If the loaded model returns a dict, extract the single tensor for comparison.
if isinstance(loaded_output, dict):
    # take first value in dict (SavedModel default output)
    loaded_output = next(iter(loaded_output.values()))

# Compare outputs using small numeric tolerance.
are_close = np.allclose(original_output, loaded_output.numpy(), atol=1e-6)

# Print comparison result and both outputs briefly.
print("Loaded model output:", loaded_output.numpy().flatten())

# Print final validation line showing persistence success.
print("Outputs match after persistence:", bool(are_close))



## **3. Model Validation Checks**

### **3.1. Output Consistency Checks**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_09/Lecture_A/image_03_01.jpg?v=1769775036" width="250">



>* Compare original and exported models on inputs
>* Check predictions and numbers match within acceptable tolerance

>* Use diverse, realistic test inputs for comparison
>* Measure differences; investigate large or systematic deviations

>* Set strict or relaxed tolerances by domain
>* Investigate large outliers to find export issues



In [ ]:
#@title Python Code - Output Consistency Checks

# This script demonstrates output consistency checks.
# We compare original and exported TensorFlow models.
# Focus is on simple beginner friendly validation.

# !pip install tensorflow.

# Import required standard libraries.
import os
import random
import numpy as np

# Import TensorFlow and Keras utilities.
import tensorflow as tf
from tensorflow import keras

# Print TensorFlow version briefly.
print("TensorFlow version:", tf.__version__)

# Set deterministic random seeds.
seed_value = 42
random.seed(seed_value)

# Set NumPy random seed for reproducibility.
np.random.seed(seed_value)

# Set TensorFlow random seed for reproducibility.
tf.random.set_seed(seed_value)

# Load MNIST dataset from Keras.
(x_train, y_train), _ = keras.datasets.mnist.load_data()

# Select a small subset for quick training.
x_train_small = x_train[:2000]

# Select corresponding labels subset.
y_train_small = y_train[:2000]

# Normalize images to range zero one.
x_train_small = x_train_small.astype("float32") / 255.0

# Add channel dimension for convolution.
x_train_small = np.expand_dims(x_train_small, axis=-1)

# Validate input shape before modeling.
assert x_train_small.shape[1:] == (28, 28, 1)

# Build a simple sequential model.
model = keras.Sequential([
    keras.layers.Input(shape=(28, 28, 1)),
    keras.layers.Conv2D(8, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(10, activation="softmax"),
])

# Compile model with standard settings.
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# Train briefly with silent output.
model.fit(
    x_train_small,
    y_train_small,
    epochs=2,
    batch_size=64,
    verbose=0,
)

# Pick a small batch for validation.
validation_inputs = x_train_small[100:110]

# Confirm validation batch shape.
assert validation_inputs.shape[0] == 10

# Run original model to get predictions.
original_outputs = model.predict(
    validation_inputs,
    verbose=0,
)

# Export model using SavedModel format.
export_dir = "mnist_export_demo.keras"

# Remove existing export directory safely.
if tf.io.gfile.exists(export_dir):
    tf.io.gfile.rmtree(export_dir)

# Save the trained model for deployment.
model.save(export_dir, include_optimizer=False)

# Load exported model from disk.
exported_model = keras.models.load_model(export_dir)

# Run exported model on same inputs.
exported_outputs = exported_model.predict(
    validation_inputs,
    verbose=0,
)

# Ensure output shapes match exactly.
assert original_outputs.shape == exported_outputs.shape

# Compute absolute differences per example.
abs_diff = np.abs(original_outputs - exported_outputs)

# Compute maximum difference across all outputs.
max_diff = float(np.max(abs_diff))

# Compute mean difference across all outputs.
mean_diff = float(np.mean(abs_diff))

# Choose a simple tolerance threshold.
tolerance = 1e-5

# Check if all differences are within tolerance.
all_close = bool(max_diff <= tolerance)

# Print summary of consistency results.
print("Max difference between outputs:", max_diff)

# Print mean absolute difference value.
print("Mean difference between outputs:", mean_diff)

# Print chosen tolerance threshold.
print("Tolerance used for comparison:", tolerance)

# Print final consistency decision.
print("Are outputs consistent within tolerance?", all_close)



### **3.2. Shape and dtype checks**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_09/Lecture_A/image_03_02.jpg?v=1769775120" width="250">



>* Check tensor shapes and dtypes match originals
>* Mismatches cause subtle bugs; verify before deployment

>* Check shapes and dtypes at model boundaries
>* Compare original and exported outputs with real inputs

>* Test dynamic shapes across realistic input ranges
>* Continuously verify dtype changes don’t break assumptions



In [ ]:
#@title Python Code - Shape and dtype checks

# This script shows simple shape and dtype checks.
# We simulate original and exported models using TensorFlow.
# Focus on comparing outputs for safety in deployment.

# Uncomment the next line if tensorflow is not installed.
# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import random
import numpy as np

# Import tensorflow and set a visible version print.
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

# Set deterministic seeds for reproducibility.
seed_value = 7
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Define a tiny original model for demonstration.
original_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(4,), dtype=tf.float32),
    tf.keras.layers.Dense(3, activation="relu"),
])

# Build the model by running one dummy batch.
dummy_input = tf.zeros((1, 4), dtype=tf.float32)
_ = original_model(dummy_input)

# Create a simple exported model copy for comparison.
exported_model = tf.keras.models.clone_model(original_model)
_ = exported_model(dummy_input)

# Define a helper to run a model safely.
def run_model_safely(model, x_batch):
    # Validate rank and last dimension before running.
    if x_batch.ndim != 2:
        raise ValueError("Input must be rank 2 batch tensor")
    if x_batch.shape[1] != 4:
        raise ValueError("Second dimension must be size four")
    return model(x_batch)


# Create a small batch of representative test inputs.
inputs = tf.constant([[1.0, 2.0, 3.0, 4.0],
                      [0.5, 0.0, -1.0, 2.0]],
                     dtype=tf.float32)

# Run both models on the same inputs.
orig_out = run_model_safely(original_model, inputs)
exp_out = run_model_safely(exported_model, inputs)

# Check and print input shape and dtype information.
print("Input shape:", inputs.shape, "dtype:", inputs.dtype)

# Check and print original model output shape and dtype.
print("Original output shape:", orig_out.shape,
      "dtype:", orig_out.dtype)

# Check and print exported model output shape and dtype.
print("Exported output shape:", exp_out.shape,
      "dtype:", exp_out.dtype)

# Compare shapes and dtypes for consistency.
shapes_match = orig_out.shape == exp_out.shape
dtypes_match = orig_out.dtype == exp_out.dtype

# Print a short validation summary for the student.
print("Shapes match?", bool(shapes_match))
print("Dtypes match?", bool(dtypes_match))

# Also check that values are numerically close enough.
max_diff = tf.reduce_max(tf.abs(orig_out - exp_out)).numpy()
print("Maximum absolute difference between outputs:", float(max_diff))




### **3.3. Debugging Export Failures**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_09/Lecture_A/image_03_03.jpg?v=1769775172" width="250">



>* Treat export issues as structured debugging tasks
>* Find dynamic behaviors and refactor model or inputs

>* Compare original and exported models on simple inputs
>* Change one factor to find failing conditions

>* Refactor dynamic or unsupported patterns into exportable forms
>* Re-export, compare outputs, and iterate for reliability



In [ ]:
#@title Python Code - Debugging Export Failures

# This script shows debugging export failures.
# We simulate export and compare model behaviors.
# Focus on simple checks and clear printed messages.

# TensorFlow is available by default in this environment.
# We only use standard library and tensorflow here.
# No extra installations are required for this script.

# Import required modules for the demonstration.
import os
import random
import numpy as np
import tensorflow as tf

# Set deterministic seeds for reproducible behavior.
seed_value = 7
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Print TensorFlow version as framework information.
print("TensorFlow version:", tf.__version__)

# Define a tiny model with a risky dynamic branch.
class DynamicBranchModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense_small = tf.keras.layers.Dense(2)
        self.dense_large = tf.keras.layers.Dense(2)

    def call(self, inputs, training=False):
        mean_value = tf.reduce_mean(inputs, axis=-1, keepdims=True)
        condition = mean_value > 0.0
        branch_small = self.dense_small(inputs)
        branch_large = self.dense_large(inputs)
        return tf.where(condition, branch_large, branch_small)


# Create a simple wrapper that mimics an exported program.
class ExportedProgramWrapper:
    def __init__(self, original_model):
        self.model = original_model

    def __call__(self, inputs):
        if inputs.shape[-1] != 3:
            raise ValueError("Exported program expects last dimension three")
        return self.model(inputs, training=False)


# Build model and exported wrapper with a dummy call.
model = DynamicBranchModel()
dummy_input = tf.zeros((1, 3), dtype=tf.float32)
_ = model(dummy_input, training=False)
exported_program = ExportedProgramWrapper(model)


# Define a helper to compare original and exported outputs.
def compare_models(inputs, description):
    original_out = model(inputs, training=False)
    exported_out = exported_program(inputs)
    diff = tf.reduce_max(tf.abs(original_out - exported_out))
    print(description, "max difference:", float(diff))


# Create a valid input that matches exported expectations.
valid_input = tf.constant([[0.1, -0.2, 0.3]], dtype=tf.float32)
print("Valid input shape:", valid_input.shape)
compare_models(valid_input, "Valid input")


# Create an invalid input to trigger a failure scenario.
invalid_input = tf.constant([[0.1, -0.2, 0.3]], dtype=tf.float32)
print("Invalid input shape:", invalid_input.shape)

# Try running exported program and catch the raised error.
try:
    _ = exported_program(invalid_input)
except ValueError as error:
    print("Caught export style error:", str(error))

# Show that original model still runs but shapes now differ.
original_invalid_out = model(invalid_input, training=False)
print("Original model output shape:", original_invalid_out.shape)



# <font color="#418FDE" size="6.5" uppercase>**Model Export**</font>


In this lecture, you learned to:
- Describe the purpose and capabilities of torch.export in PyTorch 2.10.0. 
- Export a trained nn.Module to an exported program suitable for deployment or further compilation. 
- Inspect and validate exported models to ensure they produce consistent outputs with the original model. 

In the next Lecture (Lecture B), we will go over 'Serving Models'